### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
csvfile = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(csvfile)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [48]:
#the dataframe has repeat users, SN is not a primary key
purchase_data.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,92.114103,3.050987
std,225.310896,6.659444,52.775943,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,48.000000,1.980000
50%,389.500000,22.000000,93.000000,3.150000
75%,584.250000,25.000000,139.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


## Player Count

* Display the total number of players


In [49]:
# Return number of unique Screen Names as number of players
total_players = purchase_data['SN'].nunique()
print("Assuming each screen name is a player, there are %s total players."% total_players)

Assuming each screen name is a player, there are 576 total players.


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [50]:
# Run basic calculations on the purchase_data
num_items = purchase_data['Item ID'].nunique()
avg_price = purchase_data['Price'].mean()
num_purchases = purchase_data['Purchase ID'].count()
total_revenue = purchase_data['Price'].sum()
purchase_analysis = pd.DataFrame({"Number of Unique Items": [num_items],
                                  "Average_Price": [avg_price],
                                  "Number of Purchases": [num_purchases],
                                  "Total_Revenue": total_revenue})
purchase_analysis

,Number of Unique Items,Average_Price,Number of Purchases,Total_Revenue
0,183,3.050987,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [51]:
# determine the gender demographics of the purchase_data
purchase_data.head(1)
# count and percentage of Unique Screen Names by gender
gender = pd.DataFrame(purchase_data.groupby("Gender")['SN'].nunique())
gender_renamed = gender.rename(columns={"SN": "Total Counts"})
gender_renamed['Percentage of Players'] = gender / total_players * 100

gender_renamed


,Total Counts,Percentage of Players
Gender,,
Female,81,14.062500
Male,484,84.027778
Other / Non-Disclosed,11,1.909722



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [52]:
# determine the gender demographics of the purchase_data
purchase_data.head(1)
# Select Purchase ID, Price,  SN
purchase_analysis = purchase_data[["SN", "Gender", "Price"]]
# Group By Gender
purchase_analysis_group = purchase_analysis.groupby(["Gender"])


In [53]:
# Calculate averages for purchase data by Gender Only
# Purchase Count
purchase_count = purchase_analysis_group['SN'].count()
purchase_count

#Average Purchase Price
average_purchase_price = purchase_analysis_group['Price'].mean().round(2)

#Total Purchase Value By Gender
total_purchase_value = purchase_analysis_group['Price'].sum()

# Average Total Purchase PER PERSON



In [54]:
#Return dataframe of purchase analysis by gender
purchase_analysis_gender = pd.DataFrame({"Purchase Count" : purchase_count,
                                         "Average Purchase Price" : average_purchase_price,
                                         "Total Purchase Value" : total_purchase_value})
#                                         "Avg Total Purchase per Person" : purchase_analysis_sn})

#convert column to currency
purchase_analysis_gender["Average Purchase Price"] = purchase_analysis_gender["Average Purchase Price"].map("${:.2f}".format)
purchase_analysis_gender["Total Purchase Value"] = purchase_analysis_gender["Total Purchase Value"].map("${:.2f}".format)
#purchase_analysis_gender["Avg Total Purchase per Person"] = purchase_analysis_gender["Avg Total Purchase per Person"].map("${:.2f}".format)
purchase_analysis_gender


,Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,
Female,113,$3.20,$361.94
Male,652,$3.02,$1967.64
Other / Non-Disclosed,15,$3.35,$50.19


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [55]:
#original dataframe
purchase_data.head(2)
#Create the age bins
# bins are <10, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40+
bins = [-1,9,14,19,24,29,34,39,101]
#Create names for bins  (one less than bins)
bin_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39","40+"]

purchase_data["Age Demographics"] = pd.cut(purchase_data["Age"],bins,labels=bin_names)
purchase_data_age = purchase_data.groupby("Age Demographics")
## NOTE:  THIS DOESN'T FILTER FOR UNIQUE USERS!  FIX THIS IF THERE IS TIME

In [56]:
#Calculate count and percentage for each bin
age_count = purchase_data_age['Age'].count()
percentage_of_players = age_count / total_players
age_demo = pd.DataFrame({"Age Count" : age_count, "Percentage of Players": percentage_of_players})
age_demo['Percentage of Players'] = age_demo['Percentage of Players'].map("{:,.2%}".format)

age_demo


,Age Count,Percentage of Players
Age Demographics,,
<10,23,3.99%
10-14,28,4.86%
15-19,136,23.61%
20-24,365,63.37%
25-29,101,17.53%
30-34,73,12.67%
35-39,41,7.12%
40+,13,2.26%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [57]:
# use the same bins as were use in Age Demographics
#Calculate coun and averages
purchase_count_age = purchase_data_age['Purchase ID'].count()
avg_purchase_price_age = purchase_data_age['Price'].mean()
total_purchase_value_age = purchase_data_age['Price'].sum()


age_purchase_analysis = pd.DataFrame({"Purchase Count" : purchase_count_age,
                                      "Average Purchase Price" : avg_purchase_price_age,
                                      "Total Purchase Value" : total_purchase_value_age})
#format columns
age_purchase_analysis["Average Purchase Price"] = age_purchase_analysis["Average Purchase Price"].map("${:.2f}".format)
age_purchase_analysis["Total Purchase Value"] = age_purchase_analysis["Total Purchase Value"].map("${:.2f}".format)
age_purchase_analysis

#NEED TO ADD IN THE AVG TOTAL PRICE PER PERSON

,Purchase Count,Average Purchase Price,Total Purchase Value
Age Demographics,,,
<10,23,$3.35,$77.13
10-14,28,$2.96,$82.78
15-19,136,$3.04,$412.89
20-24,365,$3.05,$1114.06
25-29,101,$2.90,$293.00
30-34,73,$2.93,$214.00
35-39,41,$3.60,$147.67
40+,13,$2.94,$38.24


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [58]:
# Group by unique SN, but no bins

purchase_data.head(2)
# Group By SN
purchase_data_SN = purchase_data.groupby(["SN"])
# Calculations
purchase_count_SN = purchase_data_SN['Purchase ID'].count()
avg_purchase_price_SN = purchase_data_SN['Price'].mean()
total_purchase_value_SN = purchase_data_SN['Price'].sum()
#create new dataframe
SN_purchase_analysis = pd.DataFrame({"Purchase Count" : purchase_count_SN,
                                      "Average Purchase Price" : avg_purchase_price_SN,
                                      "Total Purchase Value" : total_purchase_value_SN})
SN_purhase_analysis = SN_purchase_analysis.sort_values(['Total Purchase Value'], ascending = False, inplace = True)
#format columns
SN_purchase_analysis["Average Purchase Price"] = SN_purchase_analysis["Average Purchase Price"].map("${:.2f}".format)
SN_purchase_analysis["Total Purchase Value"] = SN_purchase_analysis["Total Purchase Value"].map("${:.2f}".format)
SN_purchase_analysis.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [76]:
#Create df using the original purchase data information, extract Item ID, Item Name, and Price
popular_items = purchase_data[['Item ID','Item Name','Price']]
#Groupby Item ID and Item Name
popular_items_group = popular_items.groupby(['Item ID', 'Item Name'])
# Calculations
purchase_count_pop = popular_items_group['Item ID'].count()
item_price_pop = popular_items_group['Price'].mean()
total_purchase_value_pop = popular_items_group['Price'].sum()
#create new dataframe
pop_item_analysis = pd.DataFrame({'Purchase Count' : purchase_count_pop,
                                  'Item Price' : item_price_pop,
                                  'Total Purchase Value' : total_purchase_value_pop})
pop_item_analysis = pop_item_analysis.sort_values(['Purchase Count'], ascending = False)
pop_item_analysis.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

